# Day 5

In [1]:
print('SQL Alchemy')

SQL Alchemy


In [2]:
from urllib.request import urlopen

In [3]:
url = "https://anandology.com/tmp/hello.txt"

In [4]:
urlopen(url).read()

b'Hello, world!\n'

In [5]:
urlopen(url).read().decode()

'Hello, world!\n'

In [6]:
response = urlopen(url)

In [7]:
response

In [8]:

print(response.status)

200


In [9]:
print(response.headers)

Server: nginx/1.10.3 (Ubuntu)
Date: Fri, 11 Feb 2022 04:15:13 GMT
Content-Type: text/plain
Content-Length: 14
Last-Modified: Sat, 30 Nov 2019 10:26:32 GMT
Connection: close
ETag: "5de243d8-e"
Access-Control-Allow-Origin: *
Accept-Ranges: bytes




In [10]:
!curl https://anandology.com/tmp/hello.txt

Hello, world!


In [11]:
!curl -i https://anandology.com/tmp/hello.txt

HTTP/1.1 200 OK
Server: nginx/1.10.3 (Ubuntu)
Date: Fri, 11 Feb 2022 04:15:48 GMT
Content-Type: text/plain
Content-Length: 14
Last-Modified: Sat, 30 Nov 2019 10:26:32 GMT
Connection: keep-alive
ETag: "5de243d8-e"
Access-Control-Allow-Origin: *
Accept-Ranges: bytes

Hello, world!


In [81]:
%%file wget.py
import requests
import sys
url = sys.argv[1]
filename = sys.argv[1].split('/')[-1]
response = requests.get(url)
header = response.headers
with open(filename,'w') as f:
    f.write(response.text)
    
    
# with open(filename,'rb') as image :
#     print(image)
print(f'Downloaded to {filename}')


Overwriting wget.py


In [82]:
!python wget.py https://anandology.com/tmp/hello.txt

Downloaded to hello.txt


In [83]:
!python wget.py https://anandology.com/tmp/python.png

Downloaded to python.png


In [84]:
from IPython.display import Image
Image(filename='python.png')

Working with Database Metadata

In [23]:
from sqlalchemy import create_engine

In [24]:
engine = create_engine("sqlite:///trains.db")

In [25]:
with engine.connect() as conn:
    result = conn.execute("select 2")
    print(result.all())

[(2,)]


In [26]:
q = """
    select 
        number, 
        name, 
        from_station_code, 
        to_station_code 
    from train 
    limit 5
    """
with engine.connect() as conn:
    result = conn.execute(q)
#     print(result.all()) # read all rows

    for row in result:
        print(row)

('04601', 'Jammu Tawi Udhampur Special', 'JAT', 'UHP')
('04602', 'UDHAMPUR JAMMUTAWI DMU', 'UHP', 'JAT')
('04603', 'JAT UDAHMPUR DMU', 'JAT', 'UHP')
('04604', 'UDHAMPUR JAMMUTAWI DMU', 'UHP', 'JAT')
('04728', 'Mumbai BandraT-Bikaner SF Special', 'BDTS', 'BKN')


In [39]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData, Table

In [40]:
engine = create_engine("sqlite:///trains.db")

In [41]:
metadata = MetaData(bind=engine)

In [42]:
train_table = Table('train', metadata, autoload=True)
station_table = Table('station', metadata, autoload=True)

In [43]:
from sqlalchemy import select

In [44]:
s = station_table

query = select(
        s.c.name,
    ).limit(5)

In [45]:
stations = query.execute()
for station in stations:
    print(station)

('Badhal',)
('KICHHA',)
('Sherekan',)
('Bhukarka',)
('Nohar',)


Problem: List 10 stations in Karnataka.

In [54]:
q = station_table.select().limit(5)
print(q)

SELECT station.code, station.name, station.zone, station.state, station.address, station.latitude, station.longitude 
FROM station
 LIMIT ? OFFSET ?


In [55]:
s = station_table
q = (select(
        s.c.name)
     .where(s.c.state == 'Karnataka')).limit(10)     

In [57]:
result = q.execute()
for row in result:
    print(row)

('Chikkandawadi',)
('Aliyabad',)
('Coromandel',)
('Carmelaram',)
('CHAMARAJANAGAR',)
('CHAMARAJAPURAM',)
('Chitradurg',)
('CHANNASANDRA',)
('DUROJI',)
('Dodajala Lake',)


Using count function

In [81]:
from sqlalchemy import func

In [87]:
t = train_table

q = select(func.count(t.c.number))
print(q)

SELECT count(train.number) AS count_1 
FROM train


In [88]:
result = q.execute()
for row in result:
    print(row)

(5208,)


In [89]:
q = select(
        func.count(t.c.number).label('count'))
print(q)

SELECT count(train.number) AS count 
FROM train


In [90]:
q.execute().first()

(5208,)

In [91]:
q.execute().first().count

5208

How many trains are stating from Bangalore?

In [93]:
q = (select(
        func.count(t.c.number).label('count'))
     .where(t.c.from_station_code == 'SBC'))
print(q)

SELECT count(train.number) AS count 
FROM train 
WHERE train.from_station_code = ?


In [94]:
q.execute().first().count

57

Problem: How many stations are there in Karnataka?

In [95]:
s = station_table


In [97]:
q = (select(
        func.count(s.c.state).label('count'))
     .where(s.c.state == 'Karnataka'))
print(q)

SELECT count(station.state) AS count 
FROM station 
WHERE station.state = ?


In [98]:
q.execute().first().count

298

Which stations have the most number of trains starting from there?

In [105]:
from sqlalchemy import desc

t = train_table

q = (select(
        t.c.from_station_code,
        t.c.from_station_name,
        func.count().label('count'))
     .group_by(t.c.from_station_code, t.c.from_station_name)
#      .order_by(func.count().desc())
     .order_by(desc('count'))
     .limit(10))     
print(q)

SELECT train.from_station_code, train.from_station_name, count(*) AS count 
FROM train GROUP BY train.from_station_code, train.from_station_name ORDER BY count DESC
 LIMIT ? OFFSET ?


In [106]:
result = q.execute()
for row in result:
    print(row)

('HWH', 'HOWRAH JN', 136)
('MSB', 'CHENNAI BEACH', 122)
('NDLS', 'NEW DELHI', 80)
('DLI', 'OLD DELHI', 70)
('MAS', 'CHENNAI CENTRAL', 68)
('SBC', 'BANGALORE CITY JN', 57)
('ADI', 'AHMEDABAD JN', 55)
('LPI', 'LINGAMPALLI', 54)
('HYB', 'HYDERABAD DECCAN', 53)
('NZM', 'DELHI H NIZAMUDDIN', 51)


Problem: Which states have the most number of railway stations?

In [107]:
from sqlalchemy import desc

t = station_table

q = (select(
        t.c.state,
        func.count().label('count'))
     .group_by(t.c.state)
#      .order_by(func.count().desc())
     .order_by(desc('count'))
     .limit(10))     
print(q)

SELECT station.state, count(*) AS count 
FROM station GROUP BY station.state ORDER BY count DESC
 LIMIT ? OFFSET ?


In [108]:
result = q.execute()
for row in result:
    print(row)

('', 4484)
('Uttar Pradesh', 523)
('Rajasthan', 443)
('Gujarat', 366)
('Maharashtra', 346)
('West Bengal', 332)
('Madhya Pradesh', 312)
('Karnataka', 298)
('Tamil Nadu', 236)
('Punjab', 211)
